### This program is designed to find the shortest path and the fastest path (using Euclidean Distance as a heuristic function) for a robot starting from a designated start point on the grid to a designated end point. 

In [68]:
# The class Vertex is a data structure for vertices in an undirected graph
class Vertex:
    
    # Vertex construction and variable initialization
    def __init__(self, label):
        self.label = label
        self.edge = {}
        self.layer = 0; 
        self.parent = []
        self.explored = False
        self.tried = False
        self.pred = None
    
    # Returns the degree of the vertex
    def degree(self):
        return len(self.edge)
        
    def getLayer(self):
        return self.layer
    
    def setLayer(self, layer):
        self.layer = layer
    
    def getLabel(self):
        return self.label
    
    def isTried(self):
        return self.tried
    
    def setTried(self):
        self.tried = True
    
    def getParent(self):
        return self.parent
    
    def isExplored(self):
        return self.explored
    
    def setExplored(self):
        self.explored = True
    
    def getNeighbors(self):
        return self.edge
    
    def getPredecessor(self):
        return self.pred
    
    def setPredecessor(self, pred):
        self.pred = pred
    
    def addEdge(self, vertex):
        if vertex.label not in self.edge:
            self.edge[vertex.label] = vertex
            vertex.edge[self.label] = self
        
    def addParent(self, vertex):
        self.parent.append(vertex)
        
    def removeEdge(self, vertex):
        if vertex.label in self.edge:
            del self.edge[vertex.label]
            del vertex.edge[self.label]

            
    def info(self):
        print("Vertex Label: " + self.label)
        if self.degree == 0:
            print("This vertex has no edges.")
        else:
            edges = []
            if self.getNeighbors() != None:
                for element in self.getNeighbors():
                    edges.append(element)
            print("Neighbors of this vertex: " + str(edges))
        print("Layer: " + str(self.layer))
        if(len(self.getParent()) != None):
            print("Parent: " + str(self.getParent()))
        else:
            print("This vertex has no parent.")
            
    def bfsShortestPath(self,vertex):
        self.setExplored()
        self.setLayer(0)
        queue = []
        queue.append(self)
        while len(queue) != 0:
            v = queue.pop(0)
            neighbors = v.getNeighbors()
            if neighbors != None:
                for element in neighbors:
                    if not neighbors[element].isExplored():
                        neighbors[element].setExplored()
                        queue.append(neighbors[element])
                        neighbors[element].setLayer(v.getLayer() + 1)
                    elif neighbors[element].getLayer() < v.getLayer():
                        v.addParent(neighbors[element])
    

import numpy as np
import math

class RobotPath:
    
    def __init__(self):
        self.start = None
        self.end = None
        self.obstacles = []
        self.matrix = None
        self.path = None
    
    def createGrid(self, inputFile):
        fileName = open(inputFile, "w")
        nrow = -1
        ncol = -1
        while nrow <= 0:
            nrow = int(input("Enter the number of rows for the desired grid: "))
            if nrow <= 0:
                print("Number of rows cannot be negative or zero.")
        
        while ncol <= 0:
            ncol = int(input("Enter the number of columns for the desired grid: "))
            if ncol <= 0:
                print("Number of columns cannot be negative or zero.")

    def readInput(self, filename):
        with open(filename, 'r') as info:
            for lines in info:
                if lines.startswith('n'):
                    n = lines.split()
                    self.matrix = [ [0]*int(n[3]) for i in range(int(n[1]))]
                    self.path = [ [0]*int(n[3]) for i in range(int(n[1]))]
                    
                elif lines.startswith('s'):
                    n = lines.split()
                    self.start = (int(n[1]), int(n[2]))
                    self.matrix[int(n[1])][int(n[2])] = 'S'
                elif lines.startswith('d'):
                    n = lines.split()
                    self.end = (int(n[1]), int(n[2]))
                    self.matrix[int(n[1])][int(n[2])] = 'D'
                elif lines.startswith('o'):
                    continue
                else:
                    n = lines.split()
                    self.obstacles.append((int(n[0]), int(n[1])))
                    self.matrix[int(n[0])][int(n[1])] = '*'
        
        for rows in range(len(self.path)):
            for column in range(len(self.path[rows])):
                if (rows, column) not in self.obstacles:
                    self.path[rows][column] = Vertex((rows, column))
        
        for rows in range(len(self.path)):
            for column in range(len(self.path[rows])):
                if self.path[rows][column] != 0:
                    if (not rows - 1 < 0) and (self.path[rows-1][column] != 0):
                        self.path[rows][column].addEdge(self.path[rows-1][column])
                    
                    if (not rows + 1 > len(self.path) - 1) and (self.path[rows+1][column] != 0):
                        self.path[rows][column].addEdge(self.path[rows+1][column])
                    
                    if (not column + 1 > len(self.path[rows]) - 1) and (self.path[rows][column + 1] != 0):
                        self.path[rows][column].addEdge(self.path[rows][column + 1])
                    
                    if (not column - 1 < 0) and (self.path[rows][column - 1] != 0):
                        self.path[rows][column].addEdge(self.path[rows][column - 1])
        
    def pathTracer(self, paths, option):
        for i in range(1 , len(paths) - 1):
                direction = self.getDifference(paths[i], paths[i+1], 1)
                if self.matrix[paths[i][0]][paths[i][1]] == 0:
                    self.matrix[paths[i][0]][paths[i][1]] = direction
                else:
                    if direction not in self.matrix[paths[i][0]][paths[i][1]]:
                        self.matrix[paths[i][0]][paths[i][1]] = self.sortDirection(self.matrix[paths[i][0]][paths[i][1]] + direction)                
    
    def planShortest(self):
        self.path[self.start[0]][self.start[1]].bfsShortestPath(self.path[self.end[0]][self.end[1]])
        if self.path[self.end[0]][self.end[1]].getLayer() == 0:
            return
        vertex_list = [self.path[self.end[0]][self.end[1]]]
        while len(vertex_list) > 0:
            vertex = vertex_list.pop(0)
            if len(vertex.getParent()) > 0:
                for parent in vertex.getParent():
                    if parent == self.path[self.start[0]][self.start[1]]:
                        break
                    label = self.getDifference(vertex.getLabel(), parent.getLabel(), 0)
                    if self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] == 0:
                        self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] = label
                    else:
                        if label not in self.matrix[parent.getLabel()[0]][parent.getLabel()[1]]:
                            self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] = self.sortDirection(self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] + label)
                    if not parent.isTried():
                        parent.setTried()
                        vertex_list.append(parent)
                    

    def quickPlan(self):
        start = self.path[self.start[0]][self.start[1]]
        path = []
        start.setTried()
        path.append(start.label)
        while start != self.path[self.end[0]][self.end[1]]:
            if len(start.getNeighbors()) == 0:
                return
            choices = {}
            for element in start.getNeighbors():
                if not self.path[element[0]][element[1]].isTried():
                    choices[element] = self.EuclideanDistance((element[0], element[1]),self.end)
            if len(choices) != 0:
                key = list(choices.keys())
                value = list(choices.values())
                sort = np.argsort(value)
                sorted_dict = {key[i]: value[i] for i in sort}
                newDict = self.compare(sorted_dict)
                self.path[list(newDict.keys())[0][0]][list(newDict.keys())[0][1]].setPredecessor(start)
                start = self.path[list(newDict.keys())[0][0]][list(newDict.keys())[0][1]]
                start.setTried()
                path.append(start.label)
            else:
                path.remove(start.label)
                start = start.getPredecessor()
                if start == self.path[self.start[0]][self.start[1]]:
                    return
        self.pathTracer(path,1)

    def compare(self, dic):
        if len(dic) == 1:
            return dic
        points = list(dic.keys())
        distance = list(dic.values())
        for i in range(len(distance) - 1):
            if distance[i] == distance[i+1]:
                if points[i+1][0] < points[i][0]:
                    temp = points[i+1]
                    points[i+1] = points[i]
                    points[i] = temp
                elif points[i+1][0] == points[i][0]:
                    if points[i+1][1] < points[i][1]:
                        temp = points[i+1]
                        points[i+1] = points[i]
                        points[i] = temp
                else:
                    continue
        newDict = {}
        for j in range(len(points)):
            newDict[points[j]] = distance[j]
        return newDict
    
    def EuclideanDistance(self, tuple1, tuple2):
        return math.sqrt(pow(tuple1[0]-tuple2[0],2) + pow(tuple1[1]-tuple2[1],2))
    
    def getDifference(self, tuple1, tuple2, option):
        if tuple1[0] > tuple2[0] and tuple1[1] == tuple2[1]:
            if option == 0:
                return 's'
            elif option == 1:
                return 'n'
        elif tuple1[0] < tuple2[0] and tuple1[1] == tuple2[1]:
            if option == 0:
                return 'n'
            elif option == 1:
                return 's'
        elif tuple1[0] == tuple2[0] and tuple1[1] > tuple2[1]:
            if option == 0:
                return 'e'
            elif option == 1:
                return 'w'
        elif tuple1[0] == tuple2[0] and tuple1[1] < tuple2[1]:
            if option == 0:
                return 'w'
            elif option == 1:
                return 'e'
        else:
            return
    
    def sortDirection(self, directions):
        order = {'s':1, 'n':2, 'w':3,'e':4}
        word = list(directions)
        for i in range(len(word)):
            indexSmallest = i
            for j in range(i + 1, len(word)):
                if order[word[j]] < order[word[indexSmallest]]:
                    indexSmallest = j
            temp = word[i]
            word[i] = word[indexSmallest]
            word[indexSmallest] = temp
        direction = ""
        for char in word:
            direction = direction + char
        return direction
       
    def output(self): 
        for rows in range(len(self.matrix)):
            for column in range(len(self.matrix[rows])):
                print(f"{self.matrix[rows][column]:>5}", end="")
            print()
        print()
        
    def testOutput(self, expectFile):
        with open(expectFile, 'r') as expect:
            for rows in range(len(self.matrix)):
                line = expect.readline().split()
                for column in range(len(self.matrix[rows])):
                    if str(self.matrix[rows][column]) != str(line[column]):
                        point = (rows,column)
                        print("Error at " + str(point) + " of grid.")
                        return
            print("All correct")
            return

In [66]:
r = RobotPath()
r.readInput('Grid1.txt')
r.output()

    0    0    0    *    0    0    0    0    0    0
    0    0    S    *    0    0    0    0    0    0
    0    *    *    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    0    D    0    0
    0    0    0    0    0    0    0    0    0    0



In [67]:
r.planShortest()
r.output()

(0, 2): 1
(1, 1): 1
(0, 1): 2
(1, 0): 2
(0, 0): 3
(2, 0): 3
(3, 0): 4
(4, 0): 5
(3, 1): 5
(5, 0): 6
(4, 1): 6
(3, 2): 6
(6, 0): 7
(5, 1): 7
(4, 2): 7
(3, 3): 7
(7, 0): 8
(6, 1): 8
(5, 2): 8
(4, 3): 8
(2, 3): 8
(3, 4): 8
(8, 0): 9
(7, 1): 9
(6, 2): 9
(5, 3): 9
(4, 4): 9
(2, 4): 9
(3, 5): 9
(9, 0): 10
(8, 1): 10
(7, 2): 10
(6, 3): 10
(5, 4): 10
(4, 5): 10
(1, 4): 10
(2, 5): 10
(3, 6): 10
(9, 1): 11
(8, 2): 11
(7, 3): 11
(6, 4): 11
(5, 5): 11
(0, 4): 11
(1, 5): 11
(2, 6): 11
(3, 7): 11
(9, 2): 12
(8, 3): 12
(7, 4): 12
(6, 5): 12
(0, 5): 12
(1, 6): 12
(2, 7): 12
(4, 7): 12
(3, 8): 12
(9, 3): 13
(8, 4): 13
(7, 5): 13
(0, 6): 13
(1, 7): 13
(2, 8): 13
(5, 7): 13
(4, 8): 13
(3, 9): 13
(9, 4): 14
(8, 5): 14
(0, 7): 14
(1, 8): 14
(2, 9): 14
(6, 7): 14
(5, 8): 14
(4, 9): 14
(9, 5): 15
(8, 6): 15
(0, 8): 15
(1, 9): 15
(7, 7): 15
(6, 8): 15
(5, 9): 15
(9, 6): 16
(8, 7): 16
(0, 9): 16
(7, 8): 16
(6, 9): 16
(9, 7): 17
(8, 8): 17
(7, 9): 17
(9, 8): 18
(8, 9): 18
(9, 9): 19
    0    0    0    *    0   

In [31]:
r.testOutput("11short.txt")

Error at (3, 0) of grid.


In [69]:
for i in range(1,20):
    inputFile = "Grid" + str(i) + ".txt" 
    expectFile = str(i) + "quick.txt"
    r = RobotPath()
    r.readInput(inputFile)
    r.quickPlan()
    print(str(i) + ":")
    r.testOutput(expectFile)

1:
All correct
2:
All correct
3:
All correct
4:
All correct
5:
All correct
6:
All correct
7:
All correct
8:
All correct
9:
All correct
10:
All correct
11:
All correct
12:
All correct
13:
All correct
14:
All correct
15:
All correct
16:
All correct
17:
All correct
18:
All correct
19:
All correct


In [70]:
for i in range(1,20):
    inputFile = "Grid" + str(i) + ".txt" 
    expectFile = str(i) + "short.txt"
    r = RobotPath()
    r.readInput(inputFile)
    r.planShortest()
    print(str(i) + ":")
    r.testOutput(expectFile)

1:
All correct
2:
All correct
3:
All correct
4:
All correct
5:
All correct
6:
All correct
7:
All correct
8:
All correct
9:
All correct
10:
All correct
11:
All correct
12:
All correct
13:
All correct
14:
All correct
15:
All correct
16:
All correct
17:
All correct
18:
All correct
19:
All correct


In [15]:
p = {8:'a'}
g = p.copy()
print(g)

{8: 'a'}


In [16]:
d = ("Hello" + "%d" + ".txt") % 3
print(d)

Hello3.txt


In [46]:
hello = [2]
hello.append([3,3])
print(hello)

[2, [3, 3]]
